In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 패키지 로드
import warnings
warnings.filterwarnings('ignore')

import argparse
import datetime

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
import os, random, sys, math, heapq
import time

from scipy.sparse import csr_matrix
import scipy.sparse as sp
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
from torch.nn.init import normal_
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F

import plotnine
from plotnine import *

In [3]:
# hp
class cfg:
  batch_size = 128
  epochs = 1
  l2 = 1e-4 # l2 penalty
  lr = 0.001 # learning rate
  lr_dc = 0.1 # lr decay rate
  lr_dc_epoch = [20, 40, 60, 80] # epoch which the lr decay
  patience = 5
  dim = 32 # dim of vector
  hop = 1 # num of LGCN layers
  num_negatives = 5 # num of neg instances
  alpha = 5 # weight
  topk = [25] # recommendation list
  check_epoch = 1

# Data.py

In [4]:
class Data(object):
    def __init__(self, path, num_user, num_item, status='train'):
        self.num_user = num_user
        self.num_item = num_item
        self.status = status
        if self.status == 'train':
            self.train_dict, self.prior, self.popularity, self.train_pair = self.load_train_data(path)
            self.train_pair = np.asarray(self.train_pair)
            self.train_user = [pair[0] for pair in self.train_pair]
            self.train_item = [pair[1] for pair in self.train_pair]
            self.length = len(self.train_pair)
           
            self.UserItemNet = csr_matrix((np.ones(len(self.train_user)), (self.train_user, self.train_item)),
                                          shape=(self.num_user, self.num_item))
            self.Lap_mat, self.Adj_mat = self.build_graph()
        else:
            self.test_dict, self.test_label = self.load_test_data(path)

    def load_train_data(self, path):
        data = pd.read_csv(path, header=0) # removed sep=','
        data_dict = {}
        datapair = []
        popularity = np.zeros(self.num_item)

        for i in data.itertuples():
            # user, item, rating = getattr(i, 'profile_id'), getattr(i, 'album_id'), getattr(i, 'rating') # user > profile_id / item > album_id / rating col added at main
            user = i.profile_id
            item = i.album_id
            rating = i.rating

            user, item, rating = int(user), int(item), int(rating)
            
            popularity[item] += rating
            data_dict.setdefault(user, {})
            data_dict[user][item] = 1
            datapair.append((user, item))
        prior = popularity / sum(popularity)
        random.shuffle(datapair)
        return data_dict, prior, popularity**0.75, datapair

    def load_test_data(self, path):
        data = pd.read_csv(path, header=0) # removed sep=','
        label = np.zeros((self.num_user, self.num_item))
        data_dict = {}
        for i in data.itertuples():
            user, item = getattr(i, 'profile_id'), getattr(i, 'album_id') # user > profile_id / item > album_id
            data_dict.setdefault(user, set())
            data_dict[user].add(item)
            label[user, item] = 1
        return data_dict, label

    def build_graph(self):
        print('building graph adjacency matrix')
        st = time.time()
        adj_mat = sp.dok_matrix((self.num_user + self.num_item, self.num_user + self.num_item),
                                dtype=np.float32)
        adj_mat = adj_mat.tolil()
        R = self.UserItemNet.tolil()
        adj_mat[:self.num_user, self.num_user:] = R
        adj_mat[self.num_user:, :self.num_user] = R.T
        adj_mat = adj_mat.todok()

        rowsum = np.array(adj_mat.sum(axis=1))

        d_inv = np.power(rowsum, -0.5).flatten()
        d_inv[np.isinf(d_inv)] = 0.
        d_mat = sp.diags(d_inv)

        norm_adj = d_mat.dot(adj_mat)
        norm_adj = norm_adj.dot(d_mat)
        norm_adj = norm_adj.tocsr()
        end = time.time()
        print(f"costing {end - st}s, obtained norm_mat...")

        return norm_adj, adj_mat

    def generate_batch(self, batch_size):
        n_batch = self.length // batch_size
        if self.length % batch_size != 0:
            n_batch += 1
        slices = np.split(np.arange(n_batch * batch_size), n_batch)
        slices[-1] = np.arange(self.length - batch_size, self.length)
        return slices

    def get_slices(self, index):
        pairs = self.train_pair[index]
        users, items = [], []
        for u, i in pairs:
            users.append(u)
            items.append(i)
        return users, items

def get_number_of_users_items(file):
    data = pd.read_csv(file, header=0, dtype='int') # removed sep=',', changed dtype=str > int
    return (
        data['profile_id'].max()+1 # changed from user to profile id, num_items,
        ,data['album_id'].max()+1 # changed from item to album_id
    )
            

def convert_spmat_to_sptensor(X):
    coo = X.tocoo().astype(np.float32)
    row = torch.Tensor(coo.row).long()
    col = torch.Tensor(coo.col).long()
    index = torch.stack([row, col])
    data = torch.FloatTensor(coo.data)
    return torch.sparse.FloatTensor(index, data, torch.Size(coo.shape))

def get_uninteracted_item(train_dict, unique_users, all_items):
    uninteracted_dict = {}
    num_uninter = []
    for user in unique_users:
        interacted_items = set(train_dict[user].keys())
        uninteracted_items = set(all_items) - interacted_items
        uninteracted_dict[user] = list(uninteracted_items)
        num_uninter.append(len(uninteracted_items))
    return uninteracted_dict, num_uninter

# Model.py

In [5]:
class LightGCNWithNG(nn.Module):
    def __init__(self,
                 num_users,
                 num_items,
                 g_laplace,
                 g_adj,
                 prior,
                 popularity,
                 uninter_mat,
                 num_uninter,
                 cfg,
                 device='cpu'):
        super(LightGCNWithNG, self).__init__()
        self.num_users = num_users
        self.num_items = num_items
        self.g_laplace = g_laplace
        self.g_adj = g_adj
        self.device = device

        self.dim = cfg.dim
        self.hop = cfg.hop
        self.num_negatives = cfg.num_negatives
        self.alpha = cfg.alpha
        self.prior = prior                          # (|V|,)
        self.popularity = popularity
        self.uninter_mat = uninter_mat
        self.num_uninter = num_uninter

        self.User_Emb = nn.Embedding(self.num_users, self.dim)
        nn.init.xavier_normal_(self.User_Emb.weight)
        self.Item_Emb = nn.Embedding(self.num_items, self.dim)
        nn.init.xavier_normal_(self.Item_Emb.weight)

        # LightGCN Agg
        self.global_agg = []
        for i in range(self.hop):
            agg = LightGCNAgg(self.dim)
            self.add_module('Agg_LightGCN_{}'.format(i), agg)
            self.global_agg.append(agg)

    def computer(self):
        users_emb = self.User_Emb.weight
        items_emb = self.Item_Emb.weight
        all_emb = torch.cat((users_emb, items_emb), dim=0)
        embs = [all_emb]
        for i in range(self.hop):
            aggregator = self.global_agg[i]
            x = aggregator(A=self.g_laplace, x=embs[i])
            embs.append(x)
        embs = torch.stack(embs, dim=1)
        light_out = torch.mean(embs, dim=1)
        users, items = torch.split(light_out, [self.num_users, self.num_items])
        return users, items

    def sigmoid(self,x):
        return 1/(1+np.exp(-x))

    def bns(self, users,items, ui_scores):
        batch_size = users.size(0)
        if self.device == 'cpu':
            users = users.detach().numpy()
            ui_scores = ui_scores.detach().numpy()
        else:
            users = users.cpu().detach().numpy()
            ui_scores = ui_scores.cpu().detach().numpy()
        negatives = []
        for bs in range(batch_size):
            u = users[bs]
            i = items[bs]
            rating_vector = ui_scores[bs]
            x_ui = rating_vector[i]
            negative_items = self.uninter_mat[u]

            candidate_set = np.random.choice(negative_items, size=self.num_negatives, replace=False)
            candidate_scores = [rating_vector[l] for l in candidate_set]

            # step 1 : computing info(l)
            info = np.array([1 - self.sigmoid(x_ui - x_ul) for x_ul in candidate_scores])  # O(1)
            # step 2 : computing prior probability
            p_fn = np.array([self.prior[l] for l in candidate_set])  # O(1)
            # step 3 : computing empirical distribution function (likelihood)
            F_n = np.array([np.sum(rating_vector <= x_ul) / (self.num_items+1) for x_ul in candidate_scores])  # O(|I|)
            # step 4: computing posterior probability
            unbias = (1 - F_n) * (1 - p_fn) / (1 - F_n - p_fn + 2 * F_n * p_fn)  # O(1)
            # step 5: computing conditional sampling risk
            conditional_risk = (1 - unbias) * info - self.alpha * unbias * info  # O(1)
            j = candidate_set[conditional_risk.argsort()[0]]
            negatives.append(j)
        negatives = torch.LongTensor(negatives)
        negatives = negatives.to(self.device)
        return negatives

    def forward(self, epoch, users, items):
        all_users_emb, all_items_emb = self.computer()      # |U| * d, |V| * d
        users_emb = all_users_emb[users]    # bs * d
        items_emb = all_items_emb[items]    # bs * d

        ui_scores = torch.mm(users_emb, all_items_emb.t())  # bs * |V|
        negatives = self.bns(users,items, ui_scores)  # bs
        neg_item_emb = all_items_emb[negatives]  # bs * d

        pos_scores = torch.mul(users_emb, items_emb)
        pos_scores = pos_scores.sum(dim=1)      # (bs,)
        neg_scores = torch.mul(users_emb, neg_item_emb)
        neg_scores = neg_scores.sum(dim=1)      # (bs,)

        bpr_loss = torch.mean(F.softplus(neg_scores - pos_scores))

        return bpr_loss

    def predict(self):
        all_users_emb, all_items_emb = self.computer()      # |U| * d, |V| * d
        rate_mat = torch.mm(all_users_emb, all_items_emb.t())
        return rate_mat

class LightGCNAgg(nn.Module):
    def __init__(self, hidden_size):
        super(LightGCNAgg, self).__init__()
        self.dim = hidden_size

    def forward(self, A, x):
        '''
            A: n \times n
            x: n \times d
        '''
        return torch.sparse.mm(A, x)

# main_lightGCN.py

In [6]:
def train_test(model, train_data, test_data, train_slices, optimizer, epoch):
  print('start training: ', datetime.datetime.now())
  model.train()
  total_loss = []
  for index in train_slices:
      optimizer.zero_grad()
      users, items = train_data.get_slices(index)
      users = torch.LongTensor(users).to(device)
      items = torch.LongTensor(items).to(device)

      bpr_loss = model(epoch, users, items)

      bpr_loss.backward()

      optimizer.step()

      total_loss.append(bpr_loss.item())

  print('Loss:\t%.8f\tlr:\t%0.8f' % (np.mean(total_loss), optimizer.state_dict()['param_groups'][0]['lr']))

  print('----------------')
  print('start predicting: ', datetime.datetime.now())
  
  pred_list = [] ####################
  model.eval()
  pre_dic, rec_dic, F1_dict, ndcg_dict = {}, {}, {}, {}

  #################################################
  query_user_ids = total_data['profile_id'].unique()
  full_item_ids = np.array([c for c in range(num_items)])
  
  for user_id in query_user_ids:
    with torch.no_grad(): # 기울기 계산 안 할때
      user_ids = np.full(num_items, user_id)
      user_ids = torch.LongTensor(user_ids).to(device)
      item_ids = torch.LongTensor(full_item_ids).to(device)

      eval_output = model.forward(user_ids, item_ids)
      pred_u_score = eval.output.reshape(-1)

    pred_u_idx = np.argsort(pred_u_score)[::-1]
    pred_u = full_item_ids[pred_u_idx]
    pred_list.append(list(pred_u[:25])) # 25 top-k

  pred = pd.DataFrame()
  pred['profile_id'] = query_user_ids
  pred['predicted_list'] = pred_list
  
  rets = evaluation(train_data, pred)
  #############################################
  
  rating_mat = model.predict()    # |U| * |V|
  if device == 'cpu':
      rating_mat = rating_mat.detach().numpy()
  else:
      rating_mat = rating_mat.cpu().detach().numpy()
  rating_mat = erase(rating_mat, train_data.train_dict)

  for k in cfg.topk:
      matrices = topk_eval(rating_mat, test_data.test_label, k)
      precision, recall, F1, ndcg = matrices[0], matrices[1], matrices[2], matrices[3]
      pre_dic[k] = precision
      rec_dic[k] = recall
      F1_dict[k] = F1
      ndcg_dict[k] = ndcg

  return pre_dic, rec_dic, F1_dict, ndcg_dict, pred


def erase(score, train_dict):
  for user in train_dict:
      for item in train_dict[user]:
          score[user, item] = -1000.0
  return score


def topk_eval(score, label, k):
  '''
  :param score: prediction
  :param k: number of top-k
  '''
  evaluation = [0, 0, 0, 0]
  counter = 0
  discountlist = [1 / math.log(i + 1, 2) for i in range(1, k + 1)]

  for user_no in range(score.shape[0]):
      user_score = score[user_no].tolist()
      user_label = label[user_no].tolist()
      label_count = int(sum(user_label))
      topn_recommend_score = heapq.nlargest(k, user_score)  
      topn_recommend_index = [user_score.index(i) for i in
                              topn_recommend_score]  # map(user_score.index,topn_recommend_score)
      topn_recommend_label = [user_label[i] for i in topn_recommend_index]  
      idcg = discountlist[0:label_count]

      if label_count == 0:
          counter += 1
          continue
      else:
          topk_label = topn_recommend_label[0:k]
          true_positive = sum(topk_label)
          evaluation[0] += true_positive / k  # precision
          evaluation[1] += true_positive / label_count  # recall
          evaluation[2] += 2 * true_positive / (k + label_count)  # f1
          evaluation[3] += np.dot(topk_label, discountlist[0:]) / sum(idcg)  # ndcg
  return [i / (score.shape[0] - counter) for i in evaluation]

In [7]:
def recallk(actual, predicted, k = 25):
    """ label과 prediction 사이의 recall 평가 함수 
    Args:
        actual : 실제로 본 상품 리스트
        pred : 예측한 상품 리스트
        k : 상위 몇개의 데이터를 볼지 (ex : k=5 상위 5개의 상품만 봄)
    Returns: 
        recall_k : recall@k 
    """ 
    set_actual = set(actual)
    recall_k = len(set_actual & set(predicted[:k])) / min(k, len(set_actual))
    return recall_k

def unique(sequence):
    # preserves order
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def ndcgk(actual, predicted, k = 25):
    set_actual = set(actual)
    idcg = sum([1.0 / np.log(i + 2) for i in range(min(k, len(set_actual)))])
    dcg = 0.0
    unique_predicted = unique(predicted[:k])
    for i, r in enumerate(unique_predicted):
        if r in set_actual:
            dcg += 1.0 / np.log(i + 2)
    ndcg_k = dcg / idcg
    return ndcg_k

def evaluation(gt, pred):
    """ label과 prediction 사이의 recall, coverage, competition metric 평가 함수 
    Args:
        gt : 데이터 프레임 형태의 정답 데이터 
        pred : 데이터 프레임 형태의 예측 데이터 
    Returns: 
        rets : recall, ndcg, coverage, competition metric 결과 
            ex) {'recall': 0.123024, 'ndcg': 056809, 'coverage': 0.017455, 'score': 0.106470}
    """    
    gt = gt.groupby('profile_id')['album_id'].unique().to_frame().reset_index()
    gt.columns = ['profile_id', 'actual_list']

    evaluated_data = pd.merge(pred, gt, how = 'left', on = 'profile_id')

    evaluated_data['Recall@25'] = evaluated_data.apply(lambda x: recallk(x.actual_list, x.predicted_list), axis=1)
    evaluated_data['NDCG@25'] = evaluated_data.apply(lambda x: ndcgk(x.actual_list, x.predicted_list), axis=1)

    recall = evaluated_data['Recall@25'].mean()
    ndcg = evaluated_data['NDCG@25'] .mean()

    score = 0.75*recall + 0.25*ndcg
    rets = {"recall" :recall, 
            "ndcg" :ndcg, 
            "score" :score}
    return rets

In [8]:
def init_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')

t0 = time.time()
init_seed(42)

# manually added part start
total_data = pd.read_csv('/content/drive/MyDrive/Next AI/2022 유플러스 AI Ground/data/updated_history_data.csv', encoding='utf-8') # load total file
total_data = total_data[['profile_id', 'log_time', 'album_id']].drop_duplicates(subset=['profile_id', 'album_id', 'log_time']).sort_values(by = ['profile_id', 'log_time']).reset_index(drop = True) # drop duplicates
total_data = total_data[['profile_id', 'album_id']] # use only profile_id and album_id
total_data['rating'] = 1 # rating col with 1 added
train, test = train_test_split(total_data, test_size=0.2, random_state=42) # train test split 8:2
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)
total_data.to_csv('/content/drive/MyDrive/Next AI/2022 유플러스 AI Ground/data/total.csv') # save total file
total_file = '/content/drive/MyDrive/Next AI/2022 유플러스 AI Ground/data/total.csv'
train.to_csv('/content/drive/MyDrive/Next AI/2022 유플러스 AI Ground/data/train.csv') # save train file
test.to_csv('/content/drive/MyDrive/Next AI/2022 유플러스 AI Ground/data/test.csv') # save test file
train_file = '/content/drive/MyDrive/Next AI/2022 유플러스 AI Ground/data/train.csv' # set train file path
test_file = '/content/drive/MyDrive/Next AI/2022 유플러스 AI Ground/data/test.csv' # set test file path
# manually added part end

num_users, num_items = get_number_of_users_items(total_file)
train_data = Data(train_file, num_users, num_items, status='train')
test_data = Data(test_file, num_users, num_items, status='test')

train_slices = train_data.generate_batch(cfg.batch_size)

G_Lap_tensor = convert_spmat_to_sptensor(train_data.Lap_mat)
G_Adj_tensor = convert_spmat_to_sptensor(train_data.Adj_mat)
G_Lap_tensor = G_Lap_tensor.to(device)
G_Adj_tensor = G_Adj_tensor.to(device)

uninter_mat, num_uninter = get_uninteracted_item(train_data.train_dict, train_data.train_user, list(total_data['album_id'].astype(int).unique()))

model = LightGCNWithNG(num_users, num_items, G_Lap_tensor, G_Adj_tensor, train_data.prior,train_data.popularity,
                        uninter_mat, num_uninter, cfg, device)
model = model.to(device)
print(model)

optimizer = torch.optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.l2)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=cfg.lr_dc_epoch, gamma=cfg.lr_dc)

best_result = {}
best_epoch = {}
for k in cfg.topk:
    best_result[k] = [0., 0., 0., 0.]
    best_epoch[k] = [0, 0, 0, 0]
# bad_counter = 0
for epoch in range(cfg.epochs):
    st = time.time()
    print('-------------------------------------------')
    print('epoch: ', epoch)
    pre_dic, rec_dic, F1_dict, ndcg_dict, pred = train_test(model, train_data, test_data, train_slices, optimizer, epoch)
    scheduler.step()
    for k in cfg.topk:
        if pre_dic[k] > best_result[k][0]:
            best_result[k][0] = pre_dic[k]
            best_epoch[k][0] = epoch
        if rec_dic[k] > best_result[k][1]:
            best_result[k][1] = rec_dic[k]
            best_epoch[k][1] = epoch
        if F1_dict[k] > best_result[k][2]:
            best_result[k][2] = F1_dict[k]
            best_epoch[k][2] = epoch
        if ndcg_dict[k] > best_result[k][3]:
            best_result[k][3] = ndcg_dict[k]
            best_epoch[k][3] = epoch
        print('Pre@%d:\t%0.4f\tRecall@%d:\t%0.4f\tF1@%d:\t%0.4f\tNDCG@%d:\t%0.4f\t[%0.2f s]' %
              (k, pre_dic[k], k, rec_dic[k], k, F1_dict[k], k, ndcg_dict[k], (time.time() - st)))
        
print('------------------best result-------------------')
for k in cfg.topk:
    print('Best Result: Pre@%d: %0.4f\tRecall@%d: %0.4f\tF1@%d: \t%0.4f\tNDCG@%d: \t%0.4f [%0.2f s]' %
          (k, best_result[k][0], k, best_result[k][1], k, best_result[k][2], k, best_result[k][3], (time.time() - t0)))
    print('Best Epoch: Pre@%d: %d\tRecall@%d: %d\tF1@%d: %d\tNDCG@%d: %d\t [%0.2f s]' % (
        k, best_epoch[k][0], k, best_epoch[k][1], k, best_epoch[k][2], k, best_epoch[k][3], (time.time() - t0)))
print('------------------------------------------------')
print('Run time: %0.2f s' % (time.time() - t0))

building graph adjacency matrix
costing 62.08627653121948s, obtained norm_mat...
LightGCNWithNG(
  (User_Emb): Embedding(33033, 32)
  (Item_Emb): Embedding(25917, 32)
  (Agg_LightGCN_0): LightGCNAgg()
)
-------------------------------------------
epoch:  0
start training:  2022-12-02 06:37:23.097927
Loss:	0.56860222	lr:	0.00100000
----------------
start predicting:  2022-12-02 07:38:09.472120


TypeError: ignored